In [1]:
import re
from multiprocessing import Pool

from bs4 import BeautifulSoup
from bs4.element import NavigableString as n_str

from lxml import etree

from scraping import standard_works

from text_processing import flatten


In [4]:
from time import time
for i in range(7, 12):
    start = time()
    standard_works(size=i)
    print(f"For processor size {i} the time was {start - time()}")

For processor size 7 the time was -102.7957398891449
For processor size 8 the time was -107.52112603187561
For processor size 9 the time was -100.03668522834778
For processor size 10 the time was -101.04830384254456
For processor size 11 the time was -103.03827619552612


In [3]:
%%time

corpus = standard_works()

CPU times: user 580 ms, sys: 358 ms, total: 938 ms
Wall time: 1min 27s


In [4]:
errors = corpus.xpath('//collection[@name="failures"]')

if not errors:
    print("No errors detected.")
    
else:

    failures = etree.fromstring(etree.tostring(errors[0]))

    failures = failures.xpath('//chapter')

    print(f"The number of failures was: {len(failures)}")
    print(etree.tostring(failures[0]))

No errors detected.


In [5]:
# Examines lengths of the available chapters

lengths = sorted(
    [
        (len(flatten(c)), c.attrib['source']) 
        for c in corpus.xpath('//chapter')
    ],
    key=lambda x: x[0]
)

print(
    f"The shortest chapter is {lengths[0]}"
)

The shortest chapter is (173, 'https://www.churchofjesuschrist.org/study/scriptures/ot/ps/117?lang=eng')


In [7]:
nonascii = [
    c
    for c in corpus.xpath('//chapter')
    if not flatten(c).isascii()
]

print(f"Number of nonascii chapters is: {len(nonascii)}")

if nonascii:
    print(etree.tostring(nonascii[0]))

Number of nonascii chapters is: 0


In [9]:
# Examination of a specific chapter
url = "https://www.churchofjesuschrist.org/study/scriptures/ot/title-page?lang=eng"

chapter = process_chapter(
    get(url),
    url,
    url.split('scriptures')[1].split('/'),
)[0]

print(chapter)

chapter = etree.fromstring(chapter)

verse = 6
verse = chapter.xpath(f'//verse[@number={verse}]')[0]
verse.text

b'<chapter name="/ot/title-page?lang=eng" source="https://www.churchofjesuschrist.org/study/scriptures/ot/title-page?lang=eng"><verse number="1">Translated out of the Original Tongues: and with the Former Translations Diligently Compared and Revised, by His Majesty\'s Special Command</verse><verse number="2">Authorized King James Version</verse><verse number="3">With Explanatory Notes and Cross References to the Standard Works of The Church of Jesus Christ of Latter-day Saints</verse><verse number="4">Published by The Church of Jesus Christ of Latter-day Saints</verse><verse number="5">Salt Lake City, Utah, USA</verse><verse number="6">(C) 2013 by Intellectual Reserve, Inc. All rights reserved. Source: 2015/03/24</verse></chapter>'


'(C) 2013 by Intellectual Reserve, Inc. All rights reserved. Source: 2015/03/24'